In [1]:
import sqlite3
import pandas as pd

In [2]:
pwd

'/Users/amlvt225/Code/GitHub/donkey_golf/jupyter_notebooks'

In [3]:
cd ..

/Users/amlvt225/Code/GitHub/donkey_golf


In [4]:
ls

README.md          jupyter_notebooks/ run.py
donkey_golf/       requirements.txt


In [5]:
from donkey_golf import data_utils as du

/Users/amlvt225/Code/GitHub/donkey_golf/donkey_golf


/Users/amlvt225/Code/VirtualEnvs/venv_dgolf/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
/Users/amlvt225/Code/GitHub/donkey_golf/donkey_golf/data_utils.py:28: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_sql_dict = oyaml.load(open(conf.yaml_sql_loc))


In [19]:
df = du.login_checker('po@poo.com')

In [20]:
df

,id,username,email,image_file,password


In [21]:
df['password'][0]

IndexError: index out of bounds

In [15]:
df

,id,username,email,image_file,password
0,4,poo,poo@poo.com,default.jpg,$2b$12$HLtdo2x2ANCNd96RadWEbektz6QDkqa9c1S1HZi...


In [22]:
if df.empty:
    print('yay')
else:
    print('match')

yay


In [ ]:
conn = sqlite3.connect('/home/greenman225/Code/github/donkey_golf/donkey_golf/site.db')
pd.read_sql('SELECT name FROM sqlite_master WHERE type = "table"', conn)

In [ ]:
sql = """
SELECT * FROm leaderboard
"""
pd.read_sql(sql, conn)

In [ ]:
sql = """
SELECT c.id, c.username, b.* FROM teams a
    LEFT JOIN leaderboard b
    ON a.golfer=b.player
    LEFT JOIN user c
    ON a.id=c.id
    LIMIT 10
"""
pd.read_sql(sql, conn)

In [ ]:
sql = """
SELECT a.id, c.username, sum(b.donkey_score) as donkey_score,
sum(b.missed_cut) as missed_cut 
FROM teams a
    LEFT JOIN leaderboard b
    ON a.golfer=b.player
    LEFT JOIN user c
    ON a.id=c.id
GROUP BY 1,2
"""
pd.read_sql(sql, conn)

In [ ]:
sql = """
SELECT * FROM teams
"""
pd.read_sql(sql, conn)

In [ ]:
user_df = pd.read_sql(sql, conn)

In [ ]:
user_df

In [ ]:
def aggregate_team_score():
    '''
    Pull each users aggregate score for the current tourney
    '''
    
    sql = """
    SELECT c.id, c.username, b.* FROM teams a
    LEFT JOIN leaderboard b 
    ON a.golfer=b.player
    LEFT JOIN user c 
    ON a.id=c.id
    """

    df = pd.read_sql(sql, conn)

    # Convert to par field to an integer so we can aggregate
    df['user_score'] = df['to_par'].str.replace('+','').str.replace('E','0')
    df['user_score'] = df['user_score'].astype(int)

    # Get an overall score for each user
    team_score_df = df.groupby(['id','username'])['user_score'].sum().reset_index()
    team_score_df.sort_values('user_score', inplace=True)
    
    team_score_df['rank'] = team_score_df['user_score'].rank(ascending=True, method='min').astype(int)

    return team_score_df

def determine_ties_in_scoreboard(df):
    '''
    Takes a scoreboard df and determines rank and handles ties
    '''
    df_ties = df.copy()

    # Need to create a better rank that handles ties
    df_ties['new_rank'] = df_ties['rank'].copy()

    df_ties = df_ties.groupby(['rank'])['id'].count().reset_index()
    df_ties.loc[df_ties['id'] > 1, 'new_rank'] = "T" 

    df_ties.loc[df_ties['new_rank'].notnull(), 'final_rank'] = 'T' + df_ties['rank'].astype(str)
    df_ties.loc[df_ties['new_rank'].isnull(), 'final_rank'] = df_ties['rank'].astype(str)
    
    return df_ties[['rank','final_rank']]

def pull_scoreboard():
    '''
    Pulls aggregate game score and then adds in tie info
    '''
    df = pull_game_score()
    tie_df = determine_ties_in_scoreboard(df)
    
    df = df.merge(tie_df,
        how='left',
        on='rank')
    
    return df

In [ ]:
df = pull_game_score()

In [ ]:
pull_scoreboard()

In [ ]:
df.merge(tie_df,
        how='left',
        on='rank')

In [ ]:
df_ties

In [ ]:

#+ df_ties.loc[df_ties['id'] > 1, 'rank'].astype(str)
#df_ties = df_ties[['rank', 'new_rank']].copy()

In [ ]:
df_ties

In [ ]:
df_ties.loc[df_ties['new_rank'].notnull(), 'final_rank'] = 'T' + df_ties['rank'].astype(str)

df_ties.loc[df_ties['new_rank'].isnull(), 'final_rank'] = df_ties['rank'].astype(str)

In [ ]:
df_ties

In [ ]:
df

In [ ]:
pd.read_sql("SELECT * FROM leaderboard", conn)

# Show all tables in our database 

In [ ]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type = "table"', conn)

In [ ]:
user_id=6
sql = f"""
SELECT * from teams a
INNER JOIN leaderboard b
    ON a.golfer = b.player
WHERE id = {user_id}    
"""
pd.read_sql(sql, conn)

In [ ]:
pd.read_sql('SELECT * from leaderboard', conn)

In [ ]:
pd.read_sql('SELECT * from user', conn)